# Visualizing Scalar Fields
## Rafael J. Mantilla - 201124446

We first begin by attempting to visualize the file wind_image.vti through isosurfaces:

In [43]:
import vtk

reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("./data/wind_image.vti")
reader.Update()

scalarRange = reader.GetOutput().GetPointData().GetScalars().GetRange(-1)
contoursFilter = vtk.vtkContourFilter()
contoursFilter.SetInputConnection(reader.GetOutputPort())
contoursFilter.GenerateValues(200, scalarRange)

contoursMapper = vtk.vtkPolyDataMapper()
contoursMapper.SetInputConnection(contoursFilter.GetOutputPort())
contoursMapper.SetColorModeToMapScalars()
contoursMapper.SelectColorArray('wind_speed')
contoursMapper.ScalarVisibilityOn()
contoursMapper.SetScalarRange(scalarRange)


contoursActor = vtk.vtkActor()
contoursActor.SetMapper(contoursMapper)

actor = vtk.vtkActor()
actor.SetMapper(contoursMapper)
actor.GetProperty().SetOpacity(0.1)

 
# Setup rendering
renderer = vtk.vtkRenderer()
renderer.AddActor(actor)
renderer.SetBackground(1,1,1)
renderer.ResetCamera()
 
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
 
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
 
renderWindowInteractor.SetRenderWindow(renderWindow)
renderWindowInteractor.Start()


<img src = "./data/Homework 5-1.PNG" width=500>

From the first exploration it seems we are interested in high and low wind speed areas, so we focus our opacity in the lower and higher areas:

In [45]:
import vtk

reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("./data/wind_image.vti")
reader.Update()

scalarRange = reader.GetOutput().GetPointData().GetScalars().GetRange(-1)
contoursFilter = vtk.vtkContourFilter()
contoursFilter.SetInputConnection(reader.GetOutputPort())
contoursFilter.GenerateValues(200, scalarRange)

opacityTransferFunction = vtk.vtkPiecewiseFunction()
length=scalarRange[1] - scalarRange[0]
opacityTransferFunction.AddPoint(scalarRange[0],0.2)
opacityTransferFunction.AddPoint(scalarRange[0]+length/10,0.05)
opacityTransferFunction.AddPoint(scalarRange[0]+2*length/10,0.005)
opacityTransferFunction.AddPoint(scalarRange[0]+5*length/10,0.005)
opacityTransferFunction.AddPoint(scalarRange[0]+7*length/10,0.1)
opacityTransferFunction.AddPoint(scalarRange[0]+9*length/10,0.1)
opacityTransferFunction.AddPoint(scalarRange[0]+19*length/20,0.6)
opacityTransferFunction.AddPoint(scalarRange[1],0.5)

# Create transfer mapping scalar value to color
colorTransferFunction = vtk.vtkColorTransferFunction()
colorTransferFunction.AddRGBPoint(scalarRange[0],0,0,1)
colorTransferFunction.AddRGBPoint(scalarRange[0]+length/2,0,1,0)
colorTransferFunction.AddRGBPoint(scalarRange[1],1,0,0)

# The property describes how the data will look
volumeProperty = vtk.vtkVolumeProperty()
volumeProperty.SetColor(colorTransferFunction)
volumeProperty.SetScalarOpacity(opacityTransferFunction)
volumeProperty.ShadeOn()
volumeProperty.SetInterpolationTypeToLinear()

# The mapper / ray cast function know how to render the data
#volumeMapper = vtk.vtkProjectedTetrahedraMapper()
#volumeMapper = vtk.vtkUnstructuredGridVolumeZSweepMapper()
volumeMapper = vtk.vtkGPUVolumeRayCastMapper()
#volumeMapper = vtk.vtkUnstructuredGridVolumeRayCastMapper()
volumeMapper.SetInputData(reader.GetOutput())

# The volume holds the mapper and the property and
# can be used to position/orient the volume
volume = vtk.vtkVolume()
volume.SetMapper(volumeMapper)
volume.SetProperty(volumeProperty)

##------------END VOLUME RENDERING ----------------------

#outline
outline = vtk.vtkOutlineFilter()
outline.SetInputData(reader.GetOutput())
outlineMapper = vtk.vtkPolyDataMapper()
outlineMapper.SetInputConnection(outline.GetOutputPort())
outlineActor = vtk.vtkActor()
outlineActor.SetMapper(outlineMapper)
outlineActor.GetProperty().SetColor(1,1,1)

#---------RENDERER, RENDER WINDOW, AND INTERACTOR ----------
renderer = vtk.vtkRenderer()
renderer.SetBackground(0.1, 0.1, 0.1)
renderer.AddVolume(volume)
renderer.AddActor(outlineActor)
renderer.ResetCamera()

renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindow.SetSize(500, 500)
renderWindow.Render()

iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renderWindow)
iren.Start()

<img src = "./data/Homework 5-2.PNG" width=500>

Note that we can identify a section with high speed in the middle of the picture which we will call "the eye"